<a href="https://colab.research.google.com/github/vivekkishore/Pythonbasics/blob/main/meetingclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Data Preprocessing

In [13]:
# import pandas as pd
# import numpy as np
# import seaborn
# import sklearn
# import matplotlib.pyplot as plt
# %matplotlib inline
# from pylab import rcParams
# rcParams['figure.figsize'] = 10, 8
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn import preprocessing
# from sklearn.ensemble import RandomForestClassifier
# from sklearn import tree
# from sklearn.naive_bayes import GaussianNB
# from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score, accuracy_score
# import warnings
# warnings.filterwarnings('ignore')
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.feature_extraction.text import CountVectorizer
# import re
# df=pd.read_excel('meetingtype.xlsx')
# df.head()
# X=df['meeting details']
# Y=df['meeting type']
X.head()


0    New Digital Content Form discussion, In the ca...
1    migration from Shibboleth to SAML, We will dis...
2    GLR Discussion, Could we meet for 30 minutes t...
3    Discussion on SSO progress, Scheduling this ca...
4    Learn to balance your ideas using value with F...
Name: meeting details, dtype: object